## Test MLxtend Apriori on Bread/Milk

In [6]:
import pprint
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [7]:
transactions = [
    ["Bread", "Milk"],
    ["Bread", "Diapers", "Beer", "Eggs"],
    ["Milk", "Diapers", "Beer", "Coke"],
    ["Bread", "Milk", "Diapers", "Beer"],
    ["Bread", "Milk", "Diapers", "Coke"],
]

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

df_1 = pd.DataFrame(te_ary, columns=te.columns_)
df_1

,Beer,Bread,Coke,Diapers,Eggs,Milk
0,False,True,False,False,False,True
1,True,True,False,True,True,False
2,True,False,True,True,False,True
3,True,True,False,True,False,True
4,False,True,True,True,False,True


In [8]:
res = apriori(df_1, min_support=0.6, use_colnames=True)
print(res)


   support          itemsets
0      0.6            (Beer)
1      0.8           (Bread)
2      0.8         (Diapers)
3      0.8            (Milk)
4      0.6   (Diapers, Beer)
5      0.6  (Diapers, Bread)
6      0.6     (Milk, Bread)
7      0.6   (Diapers, Milk)


## Test Spark MLlib Apriori on Bread/Milk

In [9]:
df_2 = pd.DataFrame({
    'items': transactions
})
df_2

,items
0,"[Bread, Milk]"
1,"[Bread, Diapers, Beer, Eggs]"
2,"[Milk, Diapers, Beer, Coke]"
3,"[Bread, Milk, Diapers, Beer]"
4,"[Bread, Milk, Diapers, Coke]"


In [10]:
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth

spark = SparkSession.builder \
    .appName("FPGrowthExample") \
    .master("local[*]") \
    .config("spark.driver.userClassPathFirst", "true") \
    .getOrCreate()


fp = FPGrowth(
    itemsCol="items",
    minSupport=0.6,
    minConfidence=0.5
)

model = fp.fit(df_2)

print("Frequent Itemsets:")
model.freqItemsets.show(truncate=False)

# --------------------------------------------
# 5. Display association rules
# --------------------------------------------
print("Association Rules:")
model.associationRules.show(truncate=False)

# --------------------------------------------
# 6. Display predictions (items likely to be added)
# --------------------------------------------
print("Predictions:")
model.transform(df_2).show(truncate=False)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/19 11:42:19 WARN Utils: Your hostname, MacBook-Pro-de-Thomas-2.local, resolves to a loopback address: 127.0.0.1; using 172.30.177.3 instead (on interface en0)
25/11/19 11:42:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/19 11:42:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:588)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2446)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2446)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:339)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:485)
	a